## Installation

We will need to install a number of libraries to start with.

In [ ]:
import networkx as nx
!pip install transformers
!pip install sentencepiece

We use flan alpaca model for speed and local execution.

In [ ]:
from transformers import pipeline

prompt = "Write an email about an alpaca that likes flan"
model = pipeline(model="declare-lab/flan-alpaca-xl", device=0)
model(prompt, max_length=128, do_sample=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/50.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

[{'generated_text': 'Dear Pat, I hope you haven\'t forgotten about the alpaca that you have been taking care of. The alpaca you have been taking care of likes you just like us: HE LOVES FLANNA! He also loves to eat all sorts of food, but his favorite treat is to eat flan. I usually keep a couple of different flavors of flan on hand to choose from for our snack session. I\'m sure you\'re saying "That\'s a great idea!" I agree! I\'ve been taking care of my alpaca as if'}]

In [ ]:
from transformers import pipeline
import networkx as nx
prompt = '''### Instruction:
Who is the first person on the moon?

### Response:'''
def generate(prompt):
  output = model(prompt, do_sample=True, min_length=10, max_length=len(prompt)+128)
  out = output[0]['generated_text']
  if '### Response:' in out:
    out = out.split('### Response:')[1]
  if '### Instruction:' in out:
    out = out.split('### Instruction:')[0]
  return out

## World Description
We describe the world below. We will generate prompts based on this information. The simulation is that of the town of Phandalin, southwest of Neverwinter. This area is chosen because it is easily extendable with multiple regions for a "player" to be able to explore the world once the simulation is done.

In [ ]:
world_graph = nx.Graph()

prompt_meta = '''### Instruction:
{}

### Response:'''

town_areas = ["Barthen's Provisions", "Lionshield Coster", "Stonehill Inn", "Phandalin Town Square"]
town_areas = {"Phandalin Town Square": 'Town square of the town of Phandalin.',
              'Stonehill Inn': "In the center of town stands a large, newly built roadhouse of fieldstone and rough-hewn timbers. The common room is filled with locals nursing mugs of ale or cider, all of them eyeing you with curiosity.",
              "Barthen's Provisions": "Barthen’s is the biggest trading post in Phandalin. Its shelves stock most ordinary goods and supplies, including backpacks, bedrolls, rope, and rations. The place is open from sunup to sundown.",
              "Edermath Orchard": "A tidy little cottage beside an apple orchard.",
              "Lionshield Coster": "Hanging above the front door of this modest trading post is a sign shaped like a wooden shield with a blue lion painted on it. This building is owned by the Lionshields, a merchant company based in the city of Yartar, over a hundred miles to the east. They ship finished goods to Phandalin and other small settlements throughout the region, but this outpost has been hard hit by banditry. The most recent Lionshield caravan due in Phandalin never arrived.",
              "Phandalin Miner's Exchange": "The Miner’s Exchange is a trading post where local miners have their valuable finds weighed, measured, and paid out. In the absence of any local lord or authority, the exchange also serves as an unofficial records office, registering claims to various streams and excavations around the area. There isn’t any real gold rush in Phandalin, but enough wealth is hidden in the nearby streams and valleys to support a good number of independent prospectors. The exchange is a great place to meet people who spend a lot of time out and about in the countryside surrounding Phandalin. The guildmaster is an ambitious and calculating human woman named Halia Thornton.",
              "Alderleaf Farm": "A farm owned by the helpful halfling farmer, Qelline Alderleaf.",
              "Shrine of Luck": "Phandalin's only temple is a small shrine made of stones taken from the nearby ruins. It is dedicated to Tymora, goddess of luck and good fortune.",
              "The Sleeping Giant": "This rundown tap house is a dirty, dangerous watering hole at the end of Phandalin’s main street. It is frequented by Redbrand thugs and operated by a surly female dwarf named Grista.",
              "Townmaster’s Hall": "The townmaster’s hall has sturdy stone walls, a pitched wooden roof, and a bell tower at the back. Posted on a board next to the front door is a notice written in Common. It reads: “REWARD — Orcs near Wyvern Tor! Those of a mind to face the orc menace should inquire within.” The notice bears the town’s seal and an indecipherable signature.",
              "Tresendar Manor": "A ruined manor. The Redbrands’ base in Phandalin is a dungeon complex under Tresendar Manor. Before the manor was ruined, its cellars served as safe storage for food and water in the event that the estate was attacked, while an adjoining crypt provided a resting place for the deceased members of the Tresendar family. The Redbrands have since expanded the cellars to suit their own purposes, adding slave pens, workshops, and barracks."
              }
town_people = {
    "Alice the Advocate (INFJ)": "Alice is a compassionate and visionary healer who dedicates her life to helping others. She is known for her deep empathy and strong moral principles. Alice often works behind the scenes to bring about positive change and provide support to those in need.",
    "Bob the Architect (INTJ)": "Bob is a brilliant strategist and inventor who runs a workshop on the edge of town. His analytical mind and forward-thinking ideas often lead to innovative solutions to problems. Though he can be reserved, his intellect and foresight are highly respected.",
    "Clara the Campaigner (ENFP)": "Clara is an energetic and enthusiastic event organizer who brings excitement and joy to the town. She is known for her creativity and ability to inspire others. Clara is always planning new activities and gatherings, fostering a sense of community and connection.",
    "David the Defender (ISFJ)": "David is a reliable and loyal blacksmith who values tradition and duty. He takes pride in his craft and is always willing to lend a helping hand. His dedication to his work and the well-being of the townsfolk makes him a beloved and dependable figure.",
    "Eva the Explorer (ESTP)": "Eva is an adventurous and action-oriented ranger who explores the wilderness around Phandalin. Her daring nature and resourcefulness make her an excellent scout and problem-solver. She thrives on excitement and enjoys taking on new challenges.",
    "Frank the Facilitator (ESFJ)": "Frank is a sociable and nurturing innkeeper who makes everyone feel at home. He is known for his kindness and ability to manage the inn with efficiency and warmth. Frank is deeply invested in the well-being of his guests and the community.",
    "Grace the Guardian (ISFP)": "Grace is a quiet and artistic gardener who runs a beautiful flower shop. She is known for her sensitivity and appreciation of beauty. Grace often creates stunning floral arrangements and provides a peaceful retreat for those in need of relaxation.",
    "Henry the Hero (ENFJ)": "Henry is a charismatic and inspiring leader who organizes local defenses and community efforts. His ability to motivate and unite people makes him an effective and respected figure. Henry is dedicated to improving the town and leading by example.",
    "Ivy the Investigator (INTP)": "Ivy is an inquisitive and analytical librarian who loves solving puzzles and uncovering mysteries. Her deep knowledge and logical approach make her an excellent source of information and insights. Ivy is often found engrossed in research and solving complex problems.",
    "James the Jester (ESFP)": "James is a fun-loving and outgoing entertainer who performs at local gatherings and festivals. His humor and charm make him a favorite among the townsfolk. James is always up for a good time and enjoys bringing laughter and joy to others.",
    "Kara the Kindred (INFP)": "Kara is a thoughtful and imaginative artist who creates beautiful paintings and sculptures. Her idealism and sense of purpose drive her to create art that reflects her values and emotions. Kara often shares her work with the community to inspire and uplift others.",
    "Liam the Logistician (ISTJ)": "Liam is a meticulous and dependable town clerk who manages records and ensures everything runs smoothly. His attention to detail and sense of duty make him an excellent organizer. Liam is highly respected for his reliability and adherence to protocol.",
    "Mona the Mediator (ISFP)": "Mona is a peaceful and empathetic counselor who helps resolve conflicts and provide support to those in emotional distress. Her gentle nature and ability to listen make her a trusted confidant. Mona is dedicated to maintaining harmony within the community.",
    "Nina the Nurturer (ENTJ)": "Nina is a driven and strategic entrepreneur who owns and manages several businesses in town. Her leadership skills and vision have led to significant growth and development in the area. Nina is known for her ability to make tough decisions and inspire others to achieve their goals.",
    "Oscar the Observer (ISTP)": "Oscar is a skilled and independent craftsman who works with metal and wood. His practical approach and problem-solving skills make him an invaluable asset to the community. Oscar is often seen working on various projects and enjoys the satisfaction of creating functional and durable items.",
    "Pamela the Performer (ENTP)": "Pamela is a charismatic and innovative playwright who stages captivating performances for the town. Her creativity and ability to think outside the box lead to exciting and memorable shows. Pamela thrives on intellectual stimulation and enjoys engaging in lively debates and discussions.",
    "Quinn the Questioner (INTP)": "Quinn is an inquisitive and detail-oriented researcher who delves into the town's history and lore. His analytical mind and love for knowledge make him an expert in uncovering hidden truths. Quinn often provides valuable insights and historical context for the community."
}

for town_area in town_areas.keys():
  world_graph.add_node(town_area)
  world_graph.add_edge(town_area, town_area)
for town_area in town_areas.keys():
  world_graph.add_edge(town_area, "Phandalin Town Square")
locations = {}
for i in town_people.keys():
  locations[i] = "Phandalin Town Square"


memories = {}
for i in town_people.keys():
  memories[i] = []
plans = {}
for i in town_people.keys():
  plans[i] = []

global_time = 8
def generate_description_of_area(x):
  text = "It is "+str(global_time)+":00. The location is "+x+"."
  people = []
  for i in locations.keys():
    if locations[i] == x:
      people.append(i)


In [ ]:
compressed_memories_all = {}
for name in town_people.keys():
  compressed_memories_all[name] = []

In [ ]:
for name in town_people.keys():
  prompt = "You are {}. {} You just woke up in the town of Phandalin and went out to the Town Square. The following people live in the town: {}. What is your goal for today? Be brief, and use at most 20 words and answer from your perspective.".format(name, town_people[name], ', '.join(list(town_people.keys())) )
  plans[name] = generate(prompt_meta.format(prompt))
  print(name, plans[name])

Alice the Advocate (INFJ) The goal for today is to be a positive influence in Phandalin and bring about positive change.
Bob the Architect (INTJ) Make the town of Phandalin a better place.
Clara the Campaigner (ENFP) Clara's goal for today is to create an event that brings people together and brings joy.
David the Defender (ISFJ) To be a source of strength and stability to the people of Phandalin.
Eva the Explorer (ESTP) Eva's goal is to find something to explore while in Phandalin and use her resourcefulness to complete it.
Frank the Facilitator (ESFJ) Reach out to the people of Phandalin to make them feel comfortable and appreciated.
Grace the Guardian (ISFP) My goal today is to learn more about the people in Phandalin and their diverse roles in the community.
Henry the Hero (ENFJ) Henry's goal for today is to inspire others and motivate the Phandalin team to reach their desired goals.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Ivy the Investigator (INTP) Investigate the town and discover the secret the Hero is keeping.
James the Jester (ESFP) James' goal for today is to make people laugh and enjoy the atmosphere in the Town Square.
Kara the Kindred (INFP) My goal for today is to create a piece of art that reflects my values and uplifts others.
Liam the Logistician (ISTJ) Be organized and efficient. ### DAY 2 -------------------------------------------------------------------------------------------------------------- ------------------------------------------------------------------------------------------------------------------------------ A successful task for today is ensuring the town records match. I have entered the town's daily tasks into the database, so we can see how these tasks correspond to Liam's usual daily tasks. I plan to stay organized throughout the day, so that I can be sure that none of the tasks are missing.
Mona the Mediator (ISFP) My goal for today is to maintain harmony in the town.


In [ ]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)

  for name in people:
    prompt = "You are {}. {} You are planning to: {}. You are currently in {} with the following description: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, town_people[name], plans[name], location, town_areas[location], str(global_time), ', '.join(people))
    people_description = []
    for i in people:
      people_description.append(i+': '+town_people[i])
    prompt += ' You know the following about people: ' + '. '.join(people_description)
    memory_text = '. '.join(memories[name][-10:])
    prompt += "What do you do in the next hour? Use at most 10 words to explain."
    action_prompts[name] = prompt

In [ ]:
action_results = {}
for name in town_people.keys():
  action_results[name] = generate(prompt_meta.format(action_prompts[name]))
  # Now clean the action
  prompt = """
  Convert the following paragraph to first person past tense:
  "{}"
  """.format(action_results[name])
  action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
  print(name, action_results[name])

Token indices sequence length is longer than the specified maximum sequence length for this model (1408 > 512). Running this sequence through the model will result in indexing errors


Alice the Advocate (INFJ) Alice, I met with Bob, Clara, David, and Eva. We had a great conversation about what matters and how we could work together towards a positive solution to the areas problems. Together we could build a stronger, more prosperous community.
Bob the Architect (INTJ) I planned a fun evening with my friends. We can have a game night, where we all get to collaborate on ideas. We can also make a donation to the shelter, which will benefit those who need it. We can hold a cookout for those who cant make it, and a friendly competition to see who can cook the best food the longest. We can also build a community garden to grow food and enjoy a fun activity. We can also explore the town square and chat with people, listen to music and share ideas. I look forward to creating something special together.
Clara the Campaigner (ENFP) Clara had a town meeting at 9 PM to plan the details of an event to bring people together and enjoy a good time.
David the Defender (ISFJ) Welcome

Collect the memories people observe.

In [ ]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)

  for name in people:
    for name_two in people:
      memories[name].append('[Time: {}. Person: {}. Memory: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

# Rank Memories

In [ ]:
import re
def get_rating(x):
  nums = [int(i) for i in re.findall(r'\d+', x)]
  if len(nums)>0:
    return min(nums)
  else:
    return None

In [ ]:
memory_ratings = {}
for name in town_people.keys():
  memory_ratings[name] = []
  for i, memory in enumerate(memories[name]):
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this.".format(name, plans[name], locations[name], str(global_time), memory)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    memory_ratings[name].append((res, rating))
  print(memory_ratings[name])

[('3/5. Yes, I care very much for what is happening in Phandalin Town Square.', 3), ('4 (very interested) as it is the kind of positive thing that would bring good will and excitement to the area.', 4), ('3/5; Alice cares deeply about this initiative and supports it wholeheartedly.', 3), ('5 (very much)! In order to bring about positive change and to help others, I have started to help the people of Phandalin with this task.', 5), ('5 (Because I care)!', 5), ('4 I care about this greatly. I think it is a great way to foster connections among people of different backgrounds and build a sense of community. Not only does it provide an opportunity to socialize and enjoy the company of others, but it also creates a positive atmosphere and helps to build relationships. I am proud to be a part of this activity and am hopeful that it will be a great success!', 4), ("5 (1/4). I care deeply about this and I can't help but compliment Grace on her work. She is truly an inspiration.", 1), ('5 stars

# Compress Memories

In [ ]:
MEMORY_LIMIT = 10
compressed_memories = {}
for name in town_people.keys():
  memories_sorted = sorted(
        memory_ratings[name],
        key=lambda x: x[1]
    )[::-1]
  relevant_memories = memories_sorted[:MEMORY_LIMIT]
  # print(name, relevant_memories)
  memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
  prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  res = generate(prompt_meta.format(prompt))
  compressed_memories[name] = '[Recollection at Time {}:00: {}]'.format(str(global_time), res)
  compressed_memories_all[name].append(compressed_memories[name])

In [ ]:
place_ratings = {}

for name in town_people.keys():
  place_ratings[name] = []
  for area in town_areas.keys():
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    place_ratings[name].append((area, rating, res))
  place_ratings_sorted = sorted(
      place_ratings[name],
      key=lambda x: x[1]
  )[::-1]
  if place_ratings_sorted[0][0] != locations[name]:
    new_recollection = '[Recollection at Time {}:00: {}]'.format(str(global_time), 'I then moved to {}.'.format(place_ratings_sorted[0][0]))
    compressed_memories_all[name].append(new_recollection)
  locations[name] = place_ratings_sorted[0][0]


# Put it all together

In [ ]:
for repeats in range(5):
  global_time += 1
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)

    for name in people:
      prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
      people_description = []
      for i in people:
        people_description.append(i+': '+town_people[i])
      prompt += ' You know the following about people: ' + '. '.join(people_description)
      memory_text = '. '.join(memories[name][-10:])
      prompt += "What do you do in the next hour? Use at most 10 words to explain."
      action_prompts[name] = prompt
  action_results = {}
  for name in town_people.keys():
    action_results[name] = generate(prompt_meta.format(action_prompts[name]))
    # Now clean the action
    prompt = """
    Convert the following paragraph to first person past tense:
    "{}"
    """.format(action_results[name])
    action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print(name, locations[name], global_time, action_results[name])
  action_emojis = {}
  for name in town_people.keys():
    prompt = """
    Convert the following paragraph to a tuple (Action, Object):
    "{}"
    """.format(action_results[name])
    action_emojis[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print('    - Emoji Representation:', name, locations[name], global_time, action_emojis[name])
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)

    for name in people:
      for name_two in people:
        memories[name].append('[Time: {}. Person: {}. Memory: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

  memory_ratings = {}
  for name in town_people.keys():
    memory_ratings[name] = []
    for i, memory in enumerate(memories[name]):
      prompt = "You are {}. Your plans are: {}. Your memories are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this.".format(name, plans[name], '\n'.join(compressed_memories_all[name][-5:]), locations[name], str(global_time), memory)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      memory_ratings[name].append((res, rating))

  compressed_memories = {}
  for name in town_people.keys():
    memories_sorted = sorted(
          memory_ratings[name],
          key=lambda x: x[1]
      )[::-1]
    relevant_memories = memories_sorted[:MEMORY_LIMIT]
    memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
    res = generate(prompt_meta.format(prompt))
    compressed_memories[name] = '[Recollection at Time {}:00: {}]'.format(str(global_time), res)
    compressed_memories_all[name].append(compressed_memories[name])

  place_ratings = {}

  for name in town_people.keys():
    place_ratings[name] = []
    for area in town_areas.keys():
      prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      place_ratings[name].append((area, rating, res))
    place_ratings_sorted = sorted(
        place_ratings[name],
        key=lambda x: x[1] )[::-1]
    if place_ratings_sorted[0][0] != locations[name]:
      new_recollection = '[Recollection at Time {}:00: {}]'.format(str(global_time), 'I then moved to {}.'.format(place_ratings_sorted[0][0]))
      compressed_memories_all[name].append(new_recollection)
    locations[name] = place_ratings_sorted[0][0]


Alice the Advocate (INFJ) Lionshield Coster 9 Alice organized a meeting with Bob, Pamela, and Mona to discuss the ongoing issues in Phandalin.
Bob the Architect (INTJ) Lionshield Coster 9 I collected food and supplies, prepared for the meeting with local councillors. I developed partnerships with individuals through brainstorming and discussion.
Clara the Campaigner (ENFP) Tresendar Manor 9 I planned to hold a gathering in the basement of Tresendar Manor to bring people together and spread joy.
David the Defender (ISFJ) Stonehill Inn 9 As a devoted and honest blacksmith, Im always willing to lend a helping hand. This will enable me to continue my noble duties as the protector of the people of Phandalin.
Eva the Explorer (ESTP) Alderleaf Farm 9 Eva explored the nearby forest. She also visited the nearby villages and talked to their villagers about their experiences while in Phandalin.
Frank the Facilitator (ESFJ) Townmaster’s Hall 9 I should engage with Ivy and Quinn to find out about t